In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import datetime

In [5]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [7]:
## Encode categorical vairables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [8]:
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
### Split the data into features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [11]:
## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [12]:
## Save the encoders and scaler for later use
with open('label_encoder_gender_r.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo_r.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler_r.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Regression Problem Statement

In [13]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [17]:
## Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1) ## Output layer for regression
])
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [19]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [20]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [21]:
### Train the model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


2024-09-25 22:51:11.885362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


241/250 [===========================>..] - ETA: 0s - loss: 100524.7031 - mae: 100524.7031

2024-09-25 22:51:16.155826: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 5s 8ms/step - loss: 100424.3984 - mae: 100424.3984 - val_loss: 98700.2734 - val_mae: 98700.2734
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 100343.9062 - mae: 100343.9062 - val_loss: 98552.5391 - val_mae: 98552.5391
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 100130.8672 - mae: 100130.8672 - val_loss: 98279.3438 - val_mae: 98279.3438
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 99798.9062 - mae: 99798.9062 - val_loss: 97886.4141 - val_mae: 97886.4141
Epoch 5/100
250/250 [==============================] - 2s 6ms/step - loss: 99354.4375 - mae: 99354.4375 - val_loss: 97386.4062 - val_mae: 97386.4062
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 98797.6875 - mae: 98797.6875 - val_loss: 96779.3906 - val_mae: 96779.3906
Epoch 7/100
250/250 [==============================] - 2s 6ms/step - loss: 98144.8828 - mae: 98144.8828 - val_lo

In [22]:
%load_ext tensorboard

In [25]:
%tensorboard --logdir regressionlogs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-09-25 22:56:55.116795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

In [27]:
## Evaluate the model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE :  {test_mae}')

63/63 [==============================] - 0s 4ms/step - loss: 49982.1406 - mae: 49982.1406
Test MAE :  49982.140625


In [28]:
model.save('regression_model.h5')